In [31]:
from rdkit import Chem
import pandas as pd


In [16]:
def canonicalize_smiles(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return Chem.MolToSmiles(mol, isomericSmiles=True)
    else:
        return ''

In [17]:
def get_rank(row, base, max_rank):
    for i in range(1, max_rank+1):
        if row['target'] == row['{}{}'.format(base, i)]:
            return i
    return 0

In [32]:
#data/USPTO-50K/tgt-test.txt
targets="data/USPTO-50K/tgt/tgt-test.txt"
#predictions="predictions_USPTO-50K_model_full_version_step_500000_beam10.txt"
beam_size=10
with open(targets, 'r') as f:
    targets = [''.join(line.strip().split(' ')) for line in f.readlines()]
targets = targets[:]
predictions = [[] for i in range(beam_size)]
test_df = pd.DataFrame(targets)
test_df.columns = ['target']
total = len(test_df)




In [33]:
test_df

,target
0,C1=COCCC1.COC(=O)CCC(=O)c1ccc(O)cc1O
1,COC(=O)c1cccc(C(=O)O)c1.Nc1cccnc1N
2,CC(C)(C)OC(=O)NC1CCC(C(=O)O)CC1.CNOC
3,Nc1ccc(O)cc1.O=[N+]([O-])c1ccc(Cl)nc1Cl
4,[N-]=[N+]=NCC1=CC[C@@H](c2ccc(Cl)cc2Cl)[C@H]([...
...,...
4999,Cc1cc([N+](=O)[O-])ccc1O.Nc1cc(Cl)ccn1
5000,COC(=O)c1[nH]c2cc(Cl)cc3c2c1C(CC(=O)OC(C)(C)C)CC3
5001,COc1cc(C(F)(F)F)cc(SC)c1C(=O)NC1(c2ccccc2)CC(=...
5002,C=C(C)Cn1nc(C)c(Br)c1-c1ccc(F)cc1.OO


In [35]:
with open('predictions.txt', 'r') as f:
  lines = f.readlines()
  lines = [''.join(x.strip().split()[1:]) for x in lines]
  print(lines[1])
  for i, line in enumerate(f.readlines()):
    predictions[i % beam_size].append(''.join(line.strip().split(' ')))

oc(=o)c1cc(cccc2cc3c(=o)[nh]c(n)nc3[nh]2)cs1


In [36]:
for i, preds in enumerate('predictions.txt'):
    test_df['prediction_{}'.format(i + 1)] = preds
    test_df['canonical_prediction_{}'.format(i + 1)] = test_df['prediction_{}'.format(i + 1)].apply(
    lambda x: canonicalize_smiles(x))

test_df['rank'] = test_df.apply(lambda row: get_rank(row, 'canonical_prediction_', beam_size), axis=1)
correct = 0
invalid_smiles = 0

        

In [37]:
test_df['rank'] = test_df.apply(lambda row: get_rank(row, 'canonical_prediction_', beam_size), axis=1)
correct = 0
invalid_smiles = 0
for i in range(1, beam_size+1):
    correct += (test_df['rank'] == i).sum()
    invalid_smiles += (test_df['canonical_prediction_{}'.format(i)] == '').sum()
    if invalid_smiles:
        print('Top-{}: {:.1f}% || Invalid SMILES {:.2f}%'.format(i, correct/total*100,
                                                                 invalid_smiles/(total*i)*100))
    else:
        print('Top-{}: {:.1f}%'.format(i, correct / total * 100))

Top-1: 0.0% || Invalid SMILES 100.00%
Top-2: 0.0% || Invalid SMILES 100.00%
Top-3: 0.0% || Invalid SMILES 100.00%
Top-4: 0.0% || Invalid SMILES 100.00%
Top-5: 0.0% || Invalid SMILES 100.00%
Top-6: 0.0% || Invalid SMILES 100.00%
Top-7: 0.0% || Invalid SMILES 100.00%
Top-8: 0.0% || Invalid SMILES 100.00%
Top-9: 0.0% || Invalid SMILES 100.00%
Top-10: 0.0% || Invalid SMILES 100.00%
